In [41]:
import pandas as pd

In [42]:
# === GET THE DATA ===
link = "https://www.pnnl.gov/sites/default/files/media/file/ESGC_Cost_Performance_Database_v2024.xlsx"
ESGC_raw = pd.read_excel(link, sheet_name='Database') # so we don't have to keep hitting the server during testing

In [110]:
# === START SOME BASIC FILTERING ===
ESGC = ESGC_raw

# Parameters we care about
parameters_to_keep = [
    "Total Installed Cost ($)",
    #"Total Installed Cost ($/kW)",
    #"Total Installed Cost ($/kWh)",
    "LCOS ($/kWh)", # We'll use this to filter to the lowest-cost tech
    "LCOS_Project_Life (yrs)", # For some tech this is longer than calendar life because it assumes industry-standard augmentation
    "Fixed O&M ($/kW-year)",
    #"Calendar Life (yrs)",
    #"Primary DOD (%)", # Depth of discharge. This and the rest times we ignore for a grid-scale model (vs. installation-scale).
    #"Rest After Discharge (hrs)",
    #"Rest Before Charge (hrs)",
    "RTE (%)" # round-trip efficiency
]

# Technologies we care about -- pumped storage and compressed air aren't geographically viable in Illinois (see citations in Zotero)
technologies_to_keep = [
    #"CAES",
    #"PSH",
    "Gravitational",
    "Hydrogen",
    "Lead Acid",
    "Lithium-ion LFP",
    "Lithium-ion NMC",
    "Thermal",
    "Vanadium Redox Flow",
    "Zinc"
]

# Drop the parameters and technologies that we don't need. Keep the 'Point' values which are the central estimate. Clean up columns.
ESGC = (
    ESGC[ESGC['Parameter'].isin(parameters_to_keep) & 
             ESGC['Estimate_type'].isin(['Point']) & 
             ESGC['Technology'].isin(technologies_to_keep)]
    .drop(columns=['Estimate_type', 'Parameter_category'])
)

In [111]:
# === PIVOT THE DATA TO TURN 'PARAMETERS' INTO COLUMNS ===

# Get the list of index columns
index_columns = ESGC.columns.difference(['Parameter', 'Value']).tolist()

# Pivot the DataFrame
ESGC = ESGC.pivot_table(index=index_columns, columns='Parameter', values='Value').reset_index()

In [112]:
# === MORE ADVANCED FILTERING ===

# Get unique combinations of data year, power, and duration
unique_combinations = ESGC.groupby(['Year', 'Power_MW', 'Duration_hr'])

# Initialize a list to store result
filtered_ESGC = []

# Iterate over unique combinations, find the combination with the lowest LCOS, and discard the others. Assume that at each combination of year, power, and duration, we'd pick the most economic solution.
for (year, power, duration), group in unique_combinations:
    # Find index of row with minimum LCOS
    least_cost_index = group['LCOS ($/kWh)'].idxmin()
    # Append this row to filtered list
    filtered_ESGC.append(group.loc[[least_cost_index]])

# Concatenate all DataFrames in filtered list
filtered_ESGC = pd.concat(filtered_ESGC, ignore_index=True)

In [113]:
# === ADJUST TO MATCH THE retrieve_costs OUTPUT FORMAT ===

# These are the columns that model currently utilizes
columns_needed = [
    'technology_alias',      # = "Battery"
    'techdetail',            # Concatenate 'Technology', 'Power_MW', and 'Duration_hr'
    'core_metric_variable',  # 'Year'
    'CAPEX',                 # 'Total Installed Cost ($)'
    'FOM',                   # 'Fixed O&M ($/kW-year)' 
    'VOM',                   # = 0
    'Fuel'                   # = 0
]

# These are some other columns that the storage database contains that we need to figure out how to use
other_columns = [
    "LCOS_Project_Life (yrs)", # For some tech this is longer than calendar life because it assumes industry-standard augmentation
    "RTE (%)" # round-trip efficiency
]

# Add a "carrier" column and fill with "Storage"
filtered_ESGC['technology_alias'] = 'Battery'
filtered_ESGC['techdetail'] = filtered_ESGC['Technology'] + ' ' + filtered_ESGC['Power_MW'].astype(str) + ' MW ' + filtered_ESGC['Duration_hr'].astype(str) + ' hrs'
filtered_ESGC = filtered_ESGC.rename(columns={'Year': 'core_metric_variable', 'Total Installed Cost ($)': 'CAPEX', 'Fixed O&M ($/kW-year)': 'FOM'})
filtered_ESGC[['VOM', 'Fuel']] = 0
filtered_ESGC = filtered_ESGC[columns_needed + other_columns]


In [114]:
# Show results
filtered_ESGC

Parameter,technology_alias,techdetail,core_metric_variable,CAPEX,FOM,VOM,Fuel,LCOS_Project_Life (yrs),RTE (%)
0,Battery,Gravitational 100 MW 2 hrs,2021,2.232460e+08,21.60,0,0,49.0,0.83
1,Battery,Gravitational 100 MW 4 hrs,2021,2.925700e+08,22.80,0,0,49.0,0.83
2,Battery,Gravitational 100 MW 6 hrs,2021,3.504980e+08,23.89,0,0,49.0,0.83
3,Battery,Gravitational 100 MW 8 hrs,2021,4.103140e+08,25.06,0,0,49.0,0.83
4,Battery,Gravitational 100 MW 10 hrs,2021,4.549240e+08,26.22,0,0,49.0,0.83
...,...,...,...,...,...,...,...,...,...
65,Battery,Gravitational 1000 MW 6 hrs,2030,1.850840e+09,13.94,0,0,49.0,0.84
66,Battery,Gravitational 1000 MW 8 hrs,2030,2.119720e+09,14.99,0,0,49.0,0.84
67,Battery,Gravitational 1000 MW 10 hrs,2030,2.376500e+09,16.04,0,0,49.0,0.84
68,Battery,Gravitational 1000 MW 24 hrs,2030,3.983720e+09,23.37,0,0,49.0,0.84


# Thoughts on how to handle the data in the other columns

 - **LCOS Project Life**: This is the same as the `lifetime` parameter that is defined at the `technology_alias` level. This can be added as a column to the table for all technologies.
 - **Primary DOD**, **Rest After Discharge (hrs)**, and **Rest Before Charge (hrs)**: I believe all of these can safely be ignored because we're not modeling individual batteries at real-time resolution, but groups of batteries at hourly resolution. Ignoring these variables will give slightly optimistic results.
 - **RTE (%)**: This will need to included because additional supply is needed to account for the inefficiencies. This could be included in a seperate table and only used by the model when batteries are charged and discharged. The values are factored into the LCOS which has been used to narrow technology choices.

# A note on costs

Right now `CAPEX` contains the total cost for the specific power and duration. Does this need to be converted into a per kW value? What about variable costs by duration?

Also need to confirm whether units should be in kW, MW, kWh, etc.